# ファイル操作

* カレントディレクトリをホームに変更

In [1]:
Dir.chdir(Dir.home)
# FileUtils.cd(Dir.home)

0

## ファイル/フォルダ/シンボリックリンクの作成

* Emptyという空フォルダを作成

In [2]:
Dir.mkdir("Empty")
# FileUtils.mkdir("Empty")

0

* 一時ディレクトリの場合
```Ruby
require "tmpdir"
Dir.mktmpdir # ディレクトリのパス
```

* Blankという空ファイルを作成

In [3]:
# 既にBlankが存在していたら内容が削除される  
# 削除したくない場合は, "w" を "r" や "a" にする
io=File.open("Blank","w")
io.close

* フォルダEmptyの中にファイルBlankのシンボリックリンクSymlinkを作成

In [4]:
File.symlink("../Blank","Empty/Symlink")
# FileUtils.ln_s("../Blank","Empty/Symlink")

0

- ハードリンクの場合
```Ruby
File.link("../Blank","Empty/Link")
FileUtils.ln("../Blank","Empty/Link")
```

## 書込み
* Untitled.mdというMarkdownファイルを作成して書込み

In [5]:
# 既にUntitled.mdが存在すれば上書きされる
io=File.open("Untitled.md","w")
io.write("# Header 1")
io.close

## 移動/名称変更

* フォルダEmptyをPackageに名称変更

In [6]:
FileUtils.mv("Empty","Package")
# File.rename("Empty","Package")

0

* Packageフォルダ内のSymlinkファイルをAliasに名称変更

In [7]:
FileUtils.mv("Package/Symlink","Package/Alias")
# File.rename("Package/Symlink","Package/Alias")

0

* Untitled.mdを移動して,名称変更

In [8]:
FileUtils.mv("Untitled.md","Package/Headers.md")
# File.rename("Package/Symlink","Package/Alias")

0

## 追記
* Markdownファイルに追記

In [9]:
io=File.open("Package/Headers.md","a")
io.write("\n## Header 2\n### Header 3")
io.close

## 読込み
* Markdownファイルを読込み

In [10]:
io=File.open("Package/Headers.md","r")
puts io.read
io.close

# Header 1
## Header 2
### Header 3


## 再帰的にフォルダ作成
* フォルダを一気に作成

In [11]:
FileUtils.mkdir_p("Package/Module/Submodule/Item")

["Package/Module/Submodule/Item"]

- `Dir.mkdir` の代わりに `FileUtils.mkdir_p` を使うことで,作成するフォルダItemの上位フォルダModule,Submoduleが存在していなくても,同時に生成される

## ディレクトリの内容を表示

In [12]:
Dir.entries("Package").filter! { |v| v!="." && v!=".." }
# filter で "." や ".." を取り除く

["Alias", "Module", "Headers.md"]

- ワイルドカードを使って,条件を満たすファイルを見つけ出す場合は

```Ruby
Dir.glob("*.md")
```

## ファイル/フォルダの複製
* BlankとModuleを複製

In [13]:
FileUtils.cp("Blank","Package/Blank")
FileUtils.cp_r("Package/Module","Package/Module Copy")

- `FileUtils.cp_r` を用いることで,フォルダ内の内容もまとめて複製できる

## ファイル/フォルダの削除

* ファイルBlankを削除

In [14]:
File.delete("Blank")
# File.unlink("Blank")
# FileUtils.rm("Blank")

1

* シンボリックリンクAliasを削除

In [15]:
File.delete("Package/Alias")
# File.unlink("Package/Alias")
# FileUtils.rm("Package/Alias")

1

* フォルダModuleを削除

In [16]:
FileUtils.rm_rf("Package/Module")

["Package/Module"]

- 空のフォルダは `Dir.rmdir` で削除できる。空でない場合は削除できない。  
	`FileUtils.rm_rf` を使えば,空でなくても削除できる

## 権限を確認/変更

* 状態を確認する関数

In [17]:
def check(path)

	puts File.exist?(path) ? "存在しています" : "存在していません"

	case File.ftype(path)
		when "link" then
			puts "シンボリックリンクです"
			puts "リンク先: #{File.readlink(path)}"
		when "file" then
			puts "ファイルです"
		when "directory" then
			puts "フォルダです"
	end

	puts "読込可能です" if File.readable?(path)
	puts "書込可能です" if File.writable?(path)
	puts "実行可能です" if File.executable?(path)

	puts "空です" if File.zero?(path)

end

:check

* Markdownファイルの現在の状態を確認

In [18]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
書込可能です


* 読込/書込権限を剥奪

In [19]:
File.chmod(0044,"Package/Headers.md")

1

* 状態を確認

In [20]:
check("Package/Headers.md")

存在しています
ファイルです


* 読込/実行権限を付加

In [21]:
File.chmod(0544,"Package/Headers.md")

1

* 状態を確認

In [22]:
check("Package/Headers.md")

存在しています
ファイルです
読込可能です
実行可能です


- `File.chmod` は数値により権限を指定する
	* 4: 読込可能 (Readable)
	* 2: 書込可能 (Writable)
	* 1: 実行可能 (eXecutable)

- これらの和を3つ並べて指定する。  
    1つ目はユーザの権限,2つ目はゲストの権限,3つ目はその他の者の権限である。  
	Rubyでは8進数で表記するために,前に `0` を付加する

	e.g. `File.chmod(0754,somefile)`
	* ユーザは 7 = 4+2+1 だから,読込可能,書込可能,実行可能
	* ゲストは 5 = 4 + 1 だから,読込可能,実行可能
	* その他は 4 だから,読込のみ可能